In [1]:
# for collab

import tensorflow as tf

def colab_connect():

    if tf.__version__!='2.13.0':
        %pip install tensorflow==2.13.0
        print('Stopping RUNTIME! Please run again.')
        import os
        os.kill(os.getpid(), 9)

    from google.colab import drive
    drive.mount('/content/drive/',force_remount=True)
    %cd /content/drive/My Drive/LOB/

    %pip install automodinit keras_tuner

try:
    import google.colab
    IS_COLAB = True
    colab_connect()
except:
    IS_COLAB =False

In [2]:
if IS_COLAB:
    !nohup /usr/bin/python3 /content/drive/MyDrive/LOB/test_process.py &

In [3]:
import numpy as np
from tqdm import tqdm
import keras
import datetime

import data
from utilites import DataClass

seq_len = 100

In [4]:
# Load data
if input('Quck training? (y-yes)')=='y':
    data_len=2000
else:
    data_len=None

row_data = data.load_saved_datas(data_len)
# row_data = data.load_dataset(horizon=4)
data.inspect_datas(row_data)

datasets = data.build_datasets(
    datas=row_data,
    batch_size=100,
    seq_len=seq_len,
)
(ds_train, ds_val, ds_test) =\
(datasets['train'], datasets['val'], datasets['test'])
data.inspect_datasets(datasets)

    Datas:
train     : x= (2000, 40)      | y= (2000,)        
val       : x= (2000, 40)      | y= (2000,)        
test      : x= (2000, 40)      | y= (2000,)        
    Datasets:
train     : [20, 100, 40]
val       : [20, 100, 40]
test      : [20, 100, 40]


In [5]:
# # Save data
# data.save_data(name= 'train',x= x_train,y=y_train)
# data.save_data(name= 'val',x= x_val,y=y_val)
# data.save_data(name= 'test',x= x_test,y=y_test)

In [6]:
from models import m_preln as test_model

In [7]:
# Build
model_name=''
while model_name=='':
    model_name = input('Search name')
    
pars = DataClass(test_model.PARAMETRS)
model = test_model.build_model(**pars.Info_expanded)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 40)]         0         
                                                                 
 conv1d (Conv1D)             (None, 100, 14)           1134      
                                                                 
 conv1d_1 (Conv1D)           (None, 100, 14)           406       
                                                                 
 conv1d_2 (Conv1D)           (None, 100, 14)           406       
                                                                 
 conv1d_3 (Conv1D)           (None, 100, 14)           406       
                                                                 
 conv1d_4 (Conv1D)           (None, 100, 14)           406       
                                                                 
 layer_normalization (Layer  (None, 100, 14)           28    

In [8]:
# Callbacks
log_dir = f'Temp/callbacks/{model_name}({datetime.datetime.now().strftime("%H-%M-%S--%d.%m")})'
callbacks = [
    keras.callbacks.TensorBoard(
        log_dir=log_dir,
        histogram_freq=1,
        update_freq=1,
    ),
    # tf.keras.callbacks.ModelCheckpoint(
    #     f"{save_path}/checkPoints",
    #     monitor="val_loss",
    #     verbose=0,
    #     save_best_only=False,
    #     save_weights_only=True,
    #     mode="auto",
    #     save_freq=50,
    #     options=None,
    #     initial_value_threshold=None,
    # )
]
print(callbacks, log_dir, sep='\n')

Temp/callbacks/base_model_low(16-10-40--03.10)


In [9]:
# Train
if input('Start training now? (y-yes)')=='y':
    model.fit(
        ds_train,
        epochs=20,
        validation_data=ds_val,
        callbacks=callbacks,
    )

Epoch 1/20
20/20 [==============================] - 22s 622ms/step - loss: 2.8085 - sp_acc: 0.3630 - val_loss: 2.2802 - val_sp_acc: 0.3672
Epoch 2/20
19/20 [===========================>..] - ETA: 0s - loss: 2.3223 - sp_acc: 0.3842

KeyboardInterrupt: 